# Andre Marroquin
# Serio orellana 
# Rodirgo mansilla
# carlos valladares

## 1. Situación Problemática:

En regiones agrícolas vulnerables, los periodos de **sequía** y otros **estresores bióticos/abióticos** (plagas, enfermedades, deficiencia nutricional, viento, inundación) impactan el rendimiento de los cultivos. La identificación **temprana y en terreno** del tipo y magnitud del daño permite **priorizar asistencia** y **optimizar decisiones** (riego, fertilización, control de plagas). Sin embargo, el monitoreo manual es **lento**, **costoso** y **subjetivo**.


## Problema científico:

¿Cómo **caracterizar** y **limpiar** el dataset para obtener una visión clara y confiable de la **distribución** de daños por **temporada** y **etapa de crecimiento**, identificando **patrones**, **valores atípicos** y **faltantes**, que sirvan de base para modelos posteriores de **clasificación/regresión** del daño?


## Objetivos: 

Desarrollar un **análisis exploratorio** y **pipeline de preprocesamiento** para el dataset CGIAR, que permita entender la **estructura**, **calidad** y **patrones** de los datos (daños, etapas y temporadas), facilitando fases posteriores del proyecto (modelado, evaluación y despliegue).

## 2. Investigación preliminar

- **Contexto agrícola**: La sequía reduce disponibilidad hídrica y afecta procesos fisiológicos (germinación, crecimiento vegetativo, floración), generando pérdidas.  
- **Dimensión fenológica**: Las **etapas de crecimiento (S, V, F, M)** condicionan la **susceptibilidad** y el impacto de cada daño.  
- **Tipos de daño (DR, DS, FD, G, ND, PS, WD, WN)**:  
  - **DR** (Drought), **FD** (Flood): estresores abióticos con patrones visuales diferentes.  
  - **DS** (Disease), **PS** (Pest), **WD** (Weed): bióticos y de manejo.  
  - **ND** (Nutrient Deficient): clorosis/necrosis; **WN** (Wind): daño físico; **G** (Good): referencia de buen estado.  
- **Etiquetado**: Extensión del daño en **porcentajes** (0–100 en pasos de 10).  
- **Implicación para análisis**:  
  - La **distribución por temporada** (LR/SR, 2020/2021) puede influir en patrones de daño.  
  - La **etapa** modula la **apariencia y severidad** de daños, clave para interpretar resultados y posteriormente diseñar modelos.

## 3. Análisis inicial del problema y los datos disponibles
- **Archivos**: `Train.csv` (con etiquetas), `Test.csv` (sin etiquetas), `SampleSubmission.csv` (formato de envío).  
- **Variables esperadas**:
  - Identificador de imagen (p. ej., `ID` o similar).
  - **Temporada** (p. ej., `LR2020`, `SR2021`), explícita o inferible desde el ID.
  - **Etapa de crecimiento** (`F/M/S/V` o nombre completo).
  - **Columnas de daño**: {`DR`,`DS`,`FD`,`G`,`ND`,`PS`,`WD`,`WN`} con porcentajes 0..100 (pasos de 10).
- **Riesgos de calidad**:
  - Falta de columnas explícitas (temporada/etapa).
  - Tipos de datos incorrectos (strings en columnas numéricas).
  - Etiquetas con múltiplos de 10 y ceros frecuentes → distribuciones discretas.
  - Diferencias de distribución por **temporada** que pueden sesgar análisis.



# IMPORTS Y CONFIGURACIÓN

In [15]:
# librerías
import os
import sys
import math
import json
import textwrap
import warnings
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# config global
DATA_DIR = Path(".") 
TRAIN_CSV = DATA_DIR / "Train.csv"
TEST_CSV = DATA_DIR / "Test.csv"
SAMPLE_CSV = DATA_DIR / "SampleSubmission.csv"

OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# config pandas
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

print("RUTAS:")
print(" - TRAIN:", TRAIN_CSV)
print(" - TEST:", TEST_CSV)
print(" - SAMPLE:", SAMPLE_CSV)
print("OUTPUT_DIR:", OUTPUT_DIR.resolve())


RUTAS:
 - TRAIN: Train.csv
 - TEST: Test.csv
 - SAMPLE: SampleSubmission.csv
OUTPUT_DIR: C:\Users\andre\OneDrive\Desktop\PY2-DS\outputs


# CARGA DE DATOS Y VISTAS RÁPIDAS

In [16]:
def CARGAR_CSVS():
    assert TRAIN_CSV.exists(), f"NO SE ENCONTRÓ: {TRAIN_CSV}"
    assert TEST_CSV.exists(), f"NO SE ENCONTRÓ: {TEST_CSV}"
    assert SAMPLE_CSV.exists(), f"NO SE ENCONTRÓ: {SAMPLE_CSV}"
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    sample_df = pd.read_csv(SAMPLE_CSV)
    return train_df, test_df, sample_df

TRAIN_DF, TEST_DF, SAMPLE_DF = CARGAR_CSVS()

print("SHAPES:")
print(" - TRAIN:", TRAIN_DF.shape)
print(" - TEST :", TEST_DF.shape)
print(" - SAMPLE:", SAMPLE_DF.shape)

print("\nCOLUMNAS TRAIN:")
print(list(TRAIN_DF.columns))

print("\nHEAD TRAIN:")
display(TRAIN_DF.head(3))

print("\nHEAD TEST:")
display(TEST_DF.head(3))

print("\nHEAD SAMPLE:")
display(SAMPLE_DF.head(3))


SHAPES:
 - TRAIN: (26068, 6)
 - TEST : (8663, 5)
 - SAMPLE: (8663, 2)

COLUMNAS TRAIN:
['ID', 'filename', 'growth_stage', 'damage', 'extent', 'season']

HEAD TRAIN:


,ID,filename,growth_stage,damage,extent,season
0,ID_1S8OOWQYCB,L427F01330C01S03961Rp02052.jpg,S,WD,0,SR2020
1,ID_0MD959MIZ0,L1083F00930C39S12674Ip.jpg,V,G,0,SR2021
2,ID_JRJCI4Q11V,24_initial_1_1463_1463.JPG,V,G,0,LR2020



HEAD TEST:


,ID,filename,growth_stage,damage,season
0,ID_ROOWKB90UZ,L122F00315C01S02151Rp04021.jpg,V,WD,SR2020
1,ID_PTEDRY0CYM,L1089F03254C01S08845Rp25119.jpg,F,WD,LR2021
2,ID_5WJXDV96R4,L365F01913C39S12578Rp42918.jpg,V,WD,SR2021



HEAD SAMPLE:


,ID,extent
0,ID_KJ12GE2U80,0
1,ID_W33POE3DBX,0
2,ID_1DZ7VKQTS9,0
